In [ ]:
pip install -U adapters


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.0
    Uninstalling tokenizers-0.22.0:
      Successfully uninstalled tokenizers-0.22.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.0
    Uninstalling transformers-4.56.0:
      Successfully uninstalled transformers-4.56.0


In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer
from adapters import AutoAdapterModel
import torch

# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_base')

#load base model
model = AutoAdapterModel.from_pretrained('allenai/specter2_base')

#load the adapter(s) as per the required task, provide an identifier for the adapter in load_as argument and activate it
model.load_adapter("allenai/specter2", source="hf", load_as="specter2", set_active=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

.gitattributes: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/3.59M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertAdapterModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttentionWithAdapters(
              (query): LoRALinearTorch(
                in_features=768, out_features=768, bias=True
                (shared_parameters): ModuleDict()
                (loras): ModuleDict()
              )
              (key): LoRALinearTorch(
                in_features=768, out_features=768, bias=True
                (shared_parameters): ModuleDict()
                (loras): ModuleDict()
              )
              (value): LoRALinearTorch(
             

In [ ]:
import pandas as pd

CSV_PATH = '/content/papers.csv'
df = pd.read_csv(CSV_PATH, engine='python')

In [ ]:
df.head()

,Unnamed: 0,title,id,abstract,authors,url
0,0,constraint-preserving hybrid finite element me...,1907.00084,maxwell's equations describe the evolution of ...,"['yakov berchenko-kogan', 'ari stern']",https://arxiv.org/abs/1907.00084
1,1,rewriting structured cospans,2001.09029,to support the study of compositional networks...,['daniel cicala'],https://arxiv.org/abs/2001.09029
2,2,hybridization and postprocessing in finite ele...,2008.00149,we hybridize the methods of finite element ext...,"['gerard awanou', 'maurice fabien', 'johnny gu...",https://arxiv.org/abs/2008.00149
3,3,novel approach to synchronisation of wearable ...,2107.03147,synchronisation of wireless inertial measureme...,"['andreas spilz', 'michael munz']",https://arxiv.org/abs/2107.03147
4,4,functional equivariance and conservation laws ...,2111.10042,preservation of linear and quadratic invariant...,"['robert i. mclachlan', 'ari stern']",https://arxiv.org/abs/2111.10042


In [ ]:
def generate_embeddings(texts, model, tokenizer, batch_size=32):
    all_embeddings = []
    model.eval()
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            inputs = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                return_tensors="pt",
                return_token_type_ids=False,
                max_length=512
            ).to(device)

            outputs = model(**inputs)
            # CLS token (index 0) → move to CPU and detach
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu()
            all_embeddings.append(batch_embeddings)

    # Concatenate all batches into one big tensor
    return torch.cat(all_embeddings, dim=0)

title_embeddings = generate_embeddings(df['title'].tolist(), model, tokenizer)
abstract_embeddings = generate_embeddings(df['abstract'].tolist(), model, tokenizer)

In [ ]:
from sklearn.preprocessing import normalize
import numpy as np

# L2 norm here so cosine similarity can be done with just inner product
abstract_embeddings = normalize(abstract_embeddings)
title_embeddings = normalize(title_embeddings)

np.save('abstract_embeddings.npy', abstract_embeddings)
np.save('title_embeddings.npy', title_embeddings)

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 38.7 MB/s eta 0:00:00


In [ ]:
import pickle
import faiss

# We'll use FAISS for fast retrieval in kNN
index_abstract = faiss.IndexFlatIP(abstract_embeddings.shape[1])
index_abstract.add(abstract_embeddings)

index_title = faiss.IndexFlatIP(title_embeddings.shape[1])
index_title.add(title_embeddings)

faiss.write_index(index_abstract, 'index_abstract.faiss')
faiss.write_index(index_title, 'index_title.faiss')
